# Problem Set #14

**Tc2037 Implementation of Computational Methods**

May 24, 2021.

_Authors of this notebook’s solution:_

- _Student ID and Name:_
- _Student ID and Name:_

## Concurrency

A program is said to be _concurrent_ if it may have more than one active execution context — more than one “thread of control”. Concurrency has at least three important motivations:

1. _To capture the logical structure of a problem._ Many programs, particularly servers and graphical applications, must keep track of more than one largely independent “task” at the same time. Often the simplest and most logical way to structure such a program is to represent each task with a separate thread of control.

2. _To exploit extra processors, for speed._ Long a staple of high-end servers and supercomputers, multiple processors have recently become ubiquitous in desktop and laptop machines. To use them effectively, programs must generally be written (or rewritten) with concurrency in mind.

3. _To cope with separate physical devices._ Applications that run across the Internet or a more local group of machines are inherently concurrent. Likewise, many embedded applications―the control systems of a modern automobile, for example―often have separate processors for each of several devices.

In general, we use the word _concurrent_ to characterize any system in which two or more tasks may be underway (at an unpredictable point in their execution) at the same time. Under this definition, coroutines are not concurrent, because at any given time, all but one of them is stopped at a well-known place. A concurrent system is _parallel_ if more than one task can be physically active at once; this requires more than one processor. The distinction is purely an implementation and performance issue: from a semantic point of view, there is no difference between true parallelism and the “quasiparallelism” of a system that switches between tasks at unpredictable times. A parallel system is _distributed_ if its processors are associated with people or devices that are physically separated from one another in the real world. Under these definitions, “concurrent” applies to all three motivations above. “Parallel” applies to the second and third; “distributed” applies to only the third.

\[SCOTT\] pp. 575 & 576.

## Speedup

The speed of a program is the time it takes the program to execute. This could be measured in any increment of time. _Speedup_ is defined as the time it takes a program to execute sequentially (with one processor) divided by the time it takes to execute in parallel (with many processors). The formula for speedup is:

$$
S_p = \frac{T_1}{T_p}
$$

where:

* $S_p$ is the speedup obtained from using $p$ processors.
* $T_1$ is the time it takes the program to be executed sequentially.
* $T_p$ is the time it takes the program to be executed in parallel using $p$ processors.

_Linear speedup_ or _ideal speedup_ is obtained when:

$$
S_p = p
$$

When running an algorithm with linear speedup, doubling the number of processors doubles the speed. As this is ideal, it is considered very good scalability.

\[AKHTER\] pp. 13-15.

---
## Problem 1

Numerical integration is a method of computing an approximation of the area under the curve of a function, especially when the exact integral cannot be solved. For example, the value of the constant $\pi$ can be defined by the following integral:

$$
\pi = \int_{0}^{1}\frac{4}{1+x^2}\mathit{dx}
$$

However, rather than solve this integral exactly, we can approximate the solution by use of numerical integration. The following C code is an implementation of the numerical integration midpoint rectangle rule to solve the integral just shown. To compute an approximation of the area under the curve, we must compute the area of some number of rectangles (`num_rects`) by finding the midpoint (`mid`) of each rectangle and computing the height of that rectangle (`height`), which is simply the function value at that midpoint. We add together the heights of all the rectangles (`sum`) and, once computed, we multiply the sum of the heights by the width of the rectangles (`width`) to determine the desired approximation of the total area (`area`) and the value of $\pi$.

```c
double compute_pi(long num_rects) {
    double sum = 0.0;
    double width = 1.0 / (double) num_rects;
    for (long i = 0; i < num_rects; i++) {
        double mid = (i + 0.5) * width;
        double height = 4.0 / (1.0 + mid * mid);
        sum += height;
    }
    return width * sum;
}
```

Write an equivalent Clojure function `compute-pi`. Test the function with a large value of `num-rects` that takes several seconds to compute. In theory, you should get a better approximation of $\pi$ for larger values of `num-rects`. Compare your results with the first 20 decimal places of $\pi$: 

$$
3.14159265358979323846 \ldots
$$

Have in mind that 64-bit double precision floating point numbers are accurate up to sixteen decimal places but after calculations have been done there may be some rounding errors to account for.

Afterwards, modify your code so that it uses the `pmap` function to compute its result using a specified number of threads. Measure the time with 1 and $p$ threads, where $p$ is the number of logical CPU cores in your system. Calculate the speedup $S_p$.

You can get the number of logical cores with the following expression:

In [1]:
(.availableProcessors (Runtime/getRuntime))

4

In [2]:
(defn compute-pi
  [num-rects]
  (let [width (/ 1.0 num-rects)]
    (loop [i 0
           sum 0.0]
      (if (< i num-rects)
        (let [mid (* (+ i 0.5) width)
              height (/ 4.0 (+ 1.0 (* mid mid)))]
          (recur (inc i) (+ sum height)))
        (* width sum)))))

#'user/compute-pi

In [3]:
(compute-pi 100000)

3.1415926535981615

---
## Problem 2

If you convert the number 17 to binary and hexadecimal you get 10001 and 11, respectively, which happen to be palindromes (a sequence that reads the same backward as forward). Other numbers that have this same property are: 0, 153 and 255. Let’s call these numbers _bin-hex-palindromes_.

Write a Clojure function `(count-bin-hex-palindromes n num-threads)` that counts the total number of _bin-hex-palindromes_ that are less than $2^n$. So, for example, if $n=7$ the function should return 11, because $2^7 = 128$, and there are 11 _bin-hex-palindromes_ that are less than 128 (0, 1, 3, 5, 7, 9, 15, 17, 51, 85, and 119).

The implementation must use `pmap` and consider the number of threads provided by `num-threads`, assuming that $n \ge \log_2(\texttt{num-threads})$.

Test your function with a value of $n$ that takes several seconds to compute. Measure the time with 1 and $p$ threads, where $p$ is the number of logical CPU cores in your system. Finally, compute the speedup $S_p$.

**TIP:** An easy way of converting a number to binary or hexadecimal is using the corresponding Java methods:

In [4]:
(Integer/toBinaryString 17)

"10001"

In [5]:
(Integer/toHexString 17)

"11"